<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Advanced_RAG_Hybrid_Search_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (190 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123679 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed

### Load the required Packages

In [25]:
!pip install huggingface_hub langchain_community

In [26]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub


from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os

In [27]:
### Load the PDF file

In [28]:
file_path = "/content/PraveenKumarResume.pdf"
data_file = UnstructuredPDFLoader(file_path)
docs = data_file.load()

In [29]:
print(docs[0].page_content)

Sunkaraboina Praveen Kumar

+91-9347160766 | sunkaraboinap@gmail.com | linkedin.com/in/prav | github.com/prav

EDUCATION

Jawaharlal Nehru Technological University Bachelor of Technology in Computer Science

Kukatpally,Hyderabad

2022–2026

Pragathi Junior College Intermediate

Nalgonda 2020–2022

PROJECTS

React Job Portal | Vite, react-redux, tailwindcss, jsonwebtoken, cloudinary, multer, cookie-parser

[Live][GitHub]

A full-stack web application using with Node JS as backend with React as the frontend • Authentication of Admin and Employee details via MonGoDB data. • Admin can login and create company, post jobs and accept applications • Employee Login can seamlessly view all jobs posted by different companies, apply for them , and update his profile.

AI Image Generator | Vite, react-router-dom, firebase, tailwindcss,hugging face stability/ai model [Live][GitHub]

Developed a Image generator Website with React(Vite) as the frontend. • Storage using Firebase. • User can login via C

### Split Documents and Chunking

In [30]:
# create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                          chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [31]:
chunks[0].page_content

'Sunkaraboina Praveen Kumar\n\n+91-9347160766 | sunkaraboinap@gmail.com | linkedin.com/in/prav | github.com/prav\n\nEDUCATION\n\nJawaharlal Nehru Technological University Bachelor of Technology in Computer Science\n\nKukatpally,Hyderabad\n\n2022–2026\n\nPragathi Junior College Intermediate\n\nNalgonda 2020–2022\n\nPROJECTS\n\nReact Job Portal | Vite, react-redux, tailwindcss, jsonwebtoken, cloudinary, multer, cookie-parser\n\n[Live][GitHub]\n\nA full-stack web application using with Node JS as backend with React as the frontend • Authentication of Admin and Employee details via MonGoDB data. • Admin can login and create company, post jobs and accept applications • Employee Login can seamlessly view all jobs posted by different companies, apply for them , and update his profile.'

In [32]:
# Get Embedding Model from HF via API

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

### VectorStore

In [33]:
# Vector store with the selected embedding model
vectorstore = Chroma.from_documents(chunks, embeddings)

In [34]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [35]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3

### Ensemble Retriever

In [36]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.5, 0.5])

In [37]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    model_kwargs={"temperature": 0.3,"max_new_tokens":1024},
    huggingfacehub_api_token=HF_TOKEN,
)

### Prompt Template:

In [38]:
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [39]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [40]:
chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [41]:
print(chain.invoke("What is instruction tuning?"))

Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/PraveenKumarResume.pdf'}, page_content='[Live][GitHub]\n\nDeployed Machine Learnig Projects.\n\nTECHNICAL SKILLS\n\nLanguages: Java, Python, C, SQL, JavaScript, HTML/CSS Frameworks: React, Node.js, Flask, Material-UI, GroqAPI, Next JS, Tailwind, Bootstrap Developer Tools: Git, VS Code, PyCharm, Andriod Studio, Vercel, Microsoft Office Tools, Google Colab, Render Cloud/Databases: MYSQL, MongoDB, Neon(PostGreSQL), Firebase. AIML: Machine Learning and its Algorithms, Deep Learning, NLP\n\nAchievements\n\nSolved 1000+ Problems on LeetCode and 200+ Problems on Geeks For Geeks.\n\nNPTEL Certification in PYTHON DATA STRUCTURES AND ALGORITHMS 78 Percentage\n\nNPTEL Certification in DESIGN AND ANALYS

Instruction tuning is a technique used to train pre-trained language models to learn from input (natural language descriptions of the task) and response pairs. This technique allows the models to better understand and follow instructions given to them, improving their performance on various benchmarks for both language-only and multimodal tasks. For language-only tasks, instruction tuning has been shown to improve the zero-shot and few-shot performance of models such as FLAN and InstructGPT on various

In [42]:
print(chain.invoke("Who is Praveen Kumar?"))

Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/PraveenKumarResume.pdf'}, page_content='Sunkaraboina Praveen Kumar\n\n+91-9347160766 | sunkaraboinap@gmail.com | linkedin.com/in/prav | github.com/prav\n\nEDUCATION\n\nJawaharlal Nehru Technological University Bachelor of Technology in Computer Science\n\nKukatpally,Hyderabad\n\n2022–2026\n\nPragathi Junior College Intermediate\n\nNalgonda 2020–2022\n\nPROJECTS\n\nReact Job Portal | Vite, react-redux, tailwindcss, jsonwebtoken, cloudinary, multer, cookie-parser\n\n[Live][GitHub]\n\nA full-stack web application using with Node JS as backend with React as the frontend • Authentication of Admin and Employee details via MonGoDB data. • Admin can login and create company, post jobs and accept app